In [9]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime 
import time




#Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)




#Load Master Sheet
master_sheet = client.open_by_key("1fGDYnO2M-PWqMQTBDO78qbgLC-EZKiJIly68_2D-TEA").worksheet("Master2")

index = get_as_dataframe(master_sheet, evaluate_formulas=True)
index.dropna(how='all', inplace=True)
index.reset_index(drop=True, inplace=True)




# Filter only active projects
active_projects = index[index["Active Flag"] == 1]


# === SQL Server connection details ===
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""
time.sleep(5)

# === Start Execution for Each Active Project ===
for idx, row in active_projects.iterrows():
    project_name = row["Project Name"]
    schema = row["SQL Schema Name"].strip("[]")
    # schema = str(row.get("SQL Schema Name", "")).strip("[]") if pd.notna(row.get("SQL Schema Name", "")) else None

    sheet_key = row["Google Sheet Key"]

    print(f"\n🚀 Processing Project: {project_name} | Schema: {schema}")
    
    

    try:
        sheet = client.open_by_key(sheet_key)
        AppLog_Sheet = sheet.worksheet("Supervisor Log")
        Coverage_Sheet = sheet.worksheet("Coverage")
        Attendence_Sheet = sheet.worksheet("Attendence")
        Daywise_Sheet=sheet.worksheet("Day-Wise Summary")
        
    except Exception as e:
        print(f"❌ Error opening Google Sheet for {project_name}: {e}")
        continue

        
        
    try:
        with pyodbc.connect(conn_str) as conn:

            # AppLog Query
            app_query = f'''
                SELECT a.[SupervisorAppLogId],
                       a.[ManagerId] AS [Supervisor ID],
                       a.[ManagerName] AS [Supervisor Name],
                        b.[UserName], 
                       a.[UserFullName], 
                       a.[UserDesignation], 
                       a.[LISStoreCode], 
                       a.[StoreName],
                       a.[ModuleName], 
                       a.[Count],
                       a.[Time],
                       CAST(a.[Time] AS DATE) AS LogDate 
                FROM [{database}].[{schema}].[SupervisorAppLogReport] a
                LEFT JOIN [{database}].[{schema}].[UserMaster] b
                ON a.[UserId]=b.[UserId]
                WHERE MONTH([Time]) = MONTH(GETDATE() - 1)
                AND YEAR([Time]) = YEAR(GETDATE() - 1)
                  
            '''
            AppLog = pd.read_sql(app_query, conn)
            AppLog_Sheet.batch_clear(["A:M"])
            set_with_dataframe(AppLog_Sheet, AppLog)
            print("✅ Uploaded AppLog")

            # Coverage Query
            coverage_query = f'''
                SELECT ISPStoreAttendance_NormalExportId,
                       SupervisorCode,
                       ISPCode,
                       UserName,
                       FullName,
                       Designation,
                       ClientStoreCode,
                       LISStoreCode,
                       StoreName,
                       StoreAddress,
                       Format,
                       ChainCode, 
                       ChainName,
                       VisitDate,
                       PJPStatus,
                       [CheckIn-Time],
                       [CheckOut-Time],
                       TimeDuration, 
                       AutoCheckOutStatus,
                       NoPermissionReason,
                       AttendanceType, 
                       IsAudited,
                       TRY_CONVERT(DATE, VisitDate, 105) AS DateOnly,
                       CONCAT(CONCAT(TRY_CONVERT(DATE, VisitDate, 105), UserName), LISStoreCode) AS Key1
                FROM [{database}].[{schema}].[ISPStoreAttendance_NormalExport]
                WHERE MONTH(PARSE(VisitDate AS datetime USING 'en-GB')) = MONTH(GETDATE() - 1)
                  AND YEAR = YEAR(GETDATE() - 1)
                  AND (UserName LIKE '%SUP%'
                  OR UserName LIKE '%MCR-TL%')
            '''
            Coverage = pd.read_sql(coverage_query, conn)
            Coverage_Sheet.batch_clear(["A:X"])
            set_with_dataframe(Coverage_Sheet, Coverage)
            print("✅ Uploaded Coverage")

            # Attendance Query
            attendance_query = f'''
                SELECT * FROM (
                    SELECT ISPAttendanceMasterExportId,
                           SupervisorCode, 
                           ISPCode,
                           EmployeeId, 
                           UserName,
                           FullName,
                           Designation,
                           FORMAT(TRY_CONVERT(date, [Date], 105), 'dd-MM-yyyy') as Date, 
                           VisitDate,
                           [In-Time], 
                           [Out-Time], 
                           Status,
                           ApprovalStatus,
                           ApprovalDate,
                           ApprovedBy,
                           Rejectedreason,
                           CoveredStatus,
                           MerchandisingStatus,
                           EmployeeStatus,
                           ROW_NUMBER() OVER(PARTITION BY UserName, 
                           FullName,
                           Date ORDER BY [In-Time] DESC) AS Rank,
                           CASE WHEN UserName LIKE '%SUP%' THEN 1
                                WHEN UserName LIKE '%MCR-TL%' THEN 1
                                ELSE 0 END AS SupFlag
                    FROM [{database}].[{schema}].[ISPAttendanceMasterExport]
                    WHERE [Month] = MONTH(GETDATE() - 1)
                      AND [Year] = YEAR(GETDATE() - 1)
                      AND UserName NOT LIKE '%test%'
                      AND UserName NOT LIKE '%Test%'
                ) x WHERE Rank = 1
            '''
            Attendence = pd.read_sql(attendance_query, conn)
            Attendence.drop(columns=['Rank'],inplace=True)
            Attendence.at[0, 'LastUpdated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            Attendence_Sheet.batch_clear(["A:U"])
            set_with_dataframe(Attendence_Sheet, Attendence)
            print("✅ Uploaded Attendance")
            
            
            
            
            # Day-Wise Summary
            filtered_df = Attendence[Attendence["SupFlag"] == 1].copy()
            # filtered_df["Date"] = pd.to_datetime(filtered_df["Date"], dayfirst=True).dt.strftime('%d-%m-%Y')

            # When creating Day_Wise_Summary
            filtered_df["Date"] = pd.to_datetime(filtered_df["Date"], dayfirst=True, errors="coerce", infer_datetime_format=True
            ).dt.strftime('%d-%m-%Y')


            
            Day_Wise_Summary = filtered_df[["UserName", "FullName", "Date"]].drop_duplicates().reset_index(drop=True)
            Coverage["DateOnly"] = pd.to_datetime(Coverage["DateOnly"], dayfirst=True, errors='coerce').dt.strftime('%d-%m-%Y')
            
            
            Coverage["TimeDuration"] = Coverage["TimeDuration"].astype(str).fillna("00:00")
            Coverage["TimeDuration"] = Coverage["TimeDuration"].replace(r'[^0-9:]', '00:00', regex=True)
            Coverage["TimeDuration"] = Coverage["TimeDuration"].apply(lambda x: x if len(x.split(':')) == 3 else x + ":00")
            Coverage["TimeDuration"] = pd.to_timedelta(Coverage["TimeDuration"], errors='coerce')
            Coverage["TimeDuration"] = Coverage["TimeDuration"].fillna(pd.Timedelta(seconds=0))
            
            agg_coverage = Coverage.groupby(["UserName", "DateOnly"]).agg({
                "CheckIn-Time": "min",
                "CheckOut-Time": "max",
                "TimeDuration": "sum",
                "LISStoreCode": pd.Series.nunique 
            }).reset_index()
            
            
            
            agg_coverage.rename(columns={
                "DateOnly": "Date",
                "CheckIn-Time": "First Store Login",
                "CheckOut-Time": "Last Store Logout",
                "TimeDuration": "CFT",
                "LISStoreCode": "Outlets Covered" 
            }, inplace=True)
            
            agg_coverage["CFT"] = agg_coverage["CFT"].apply(
                lambda x: f"{int(x.total_seconds() // 3600):02d}:{int((x.total_seconds() % 3600) // 60):02d}"
            )
            
            Day_Wise_Summary = Day_Wise_Summary.merge(agg_coverage, on=["UserName", "Date"], how="left")
            
            Day_Wise_Summary = Day_Wise_Summary.fillna("-")
            
            # Step 10: Parse 'First Store Login' and 'Last Store Logout' to datetime using only time
            def to_time_obj(t):
                try:
                    if t == "-" or pd.isnull(t):
                        return pd.NaT
                    return pd.to_datetime(t, format="%H:%M:%S", errors='coerce')
                except:
                    return pd.NaT
            
            Day_Wise_Summary["FirstTimeParsed"] = Day_Wise_Summary["First Store Login"].apply(to_time_obj)
            Day_Wise_Summary["LastTimeParsed"] = Day_Wise_Summary["Last Store Logout"].apply(to_time_obj)
            
            
            def calc_fos(row):
                start = row["FirstTimeParsed"]
                end = row["LastTimeParsed"]
                first_login = row["First Store Login"]
            
                if first_login == "-" or pd.isnull(start) or pd.isnull(end):
                    return "-"
                diff = end - start
                if diff.total_seconds() >= 0:
                    minutes = int(diff.total_seconds() // 60)
                    return f"{minutes // 60:02d}:{minutes % 60:02d}"
                return "00:00"
            
            Day_Wise_Summary["FOS"] = Day_Wise_Summary.apply(calc_fos, axis=1)
            
            Day_Wise_Summary.drop(columns=["FirstTimeParsed", "LastTimeParsed"], inplace=True)
            
            desired_order = [ "UserName", "FullName","Date","First Store Login",
                              "Last Store Logout","CFT","FOS","Outlets Covered"]
            
            
            Day_Wise_Summary = Day_Wise_Summary[desired_order]
            
            Day_Wise_Summary = Day_Wise_Summary.sort_values("Date").reset_index(drop=True)
            
            Daywise_Sheet.batch_clear(["A:H"])
            set_with_dataframe(Daywise_Sheet, Day_Wise_Summary)
            print("✅ Uploaded Day Wise Summary")
            
            
            now_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  
            index.at[idx, "Last Run Time"] = now_str          
            print(f"🕒 Updated Last Run Time for {project_name}: {now_str}")
            
                        
                        
            

    except Exception as e:
        print(f"❌ Error processing {project_name}: {e}")
        
        

# === Write Updated Index (Master) Sheet Back ===
row_number = idx + 2  # +2 because header is row 1
col_number = index.columns.get_loc("Last Run Time") + 1

master_sheet.update_cell(row_number, col_number, now_str)

print("\n✅ Master sheet updated with Last Run Times.")




🚀 Processing Project: Piramal | Schema: PiramalManagement


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_37896\837614439.py:100: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  AppLog = pd.read_sql(app_query, conn)


✅ Uploaded AppLog


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_37896\837614439.py:137: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Coverage = pd.read_sql(coverage_query, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_37896\837614439.py:177: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Attendence = pd.read_sql(attendance_query, conn)


✅ Uploaded Coverage
✅ Uploaded Attendance


...

✅ Uploaded Coverage


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_37896\837614439.py:177: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Attendence = pd.read_sql(attendance_query, conn)


✅ Uploaded Attendance


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_37896\837614439.py:192: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  filtered_df["Date"] = pd.to_datetime(filtered_df["Date"], dayfirst=True, errors="coerce", infer_datetime_format=True


✅ Uploaded Day Wise Summary
🕒 Updated Last Run Time for Abbott: 2026-01-13 17:20:50

✅ Master sheet updated with Last Run Times.
